In [9]:
import pandas as pd
import numpy as np

In [10]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

## 데이터 수정

In [11]:
for df in [df_train,df_test]:
    # df['date'] = df['date'].apply(lambda x: x[0:8])
    # 일짜까지 되있는걸 월까지
    df['date'] = df['date'].apply(lambda x: x[0:6])
    df['yr_renovated'] = df['yr_renovated'].apply(lambda x: np.nan if x == 0 else x)
    df['yr_renovated'] = df['yr_renovated'].fillna(df['yr_built'])
for df in [df_train]:
    # 1000단위 대로 반올림
    df['price'] = df['price'].apply(lambda x: round(x, -3))

In [12]:
for df in [df_train,df_test]:
    df['total_rooms'] = df['bedrooms'] + df['bathrooms']
    df['sqft_ratio'] = df['sqft_living'] / df['sqft_lot']
    df['sqft_total_size'] = df['sqft_above'] + df['sqft_basement']
    df['sqft_ratio15'] = df['sqft_living15'] / df['sqft_lot15'] 
    df['is_renovated'] = df['yr_renovated'] - df['yr_built']
    df['is_renovated'] = df['is_renovated'].apply(lambda x: 0 if x == 0 else 1)
    df['date'] = df['date'].astype('int')

### 월별 + ZIPCODE를 활용한 더미변수 추가하여 테스트
위도 + 경도로 위치값을 대신하고 싶었지만 어떻게 구역을 나눠야 할지 모르기때문에 ZIPCODE로 대신함

In [13]:
df_train_length = len(df_train)
concat_dataset = pd.concat(objs=[df_train, df_test], axis=0)
dataset_preprocessed = pd.get_dummies(concat_dataset, columns=['date', 'zipcode'])
df_train = dataset_preprocessed[:df_train_length]
df_test = dataset_preprocessed[df_train_length:]

C:\Users\PC\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


## XGboost

In [14]:
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
import xgboost as xgb
from xgboost import XGBClassifier

In [15]:
#train_columns = [c for c in df_train.columns if c not in ['id','price','per_price','lat','long']]
train_columns = [c for c in df_train.columns if c not in ['id','price','per_price','date', 'zipcode']]
y_reg = df_train['price']

In [ ]:
xgb.cv

In [ ]:
model= xgb.XGBRegressor(
    learning_rate=0.005,
    # 모형 갯수 default=100
    n_estimators=4000,
    random_state=2019,
    num_boost_round=5000, 
    early_stopping_rounds=4000,
    verbose_eval=3000, 
    show_stdv=False,
    max_depth=100, 
    nthread=6,
    nfold=10
)

In [ ]:
model= xgb.XGBRegressor(
    learning_rate=0.005,
    # 모형 갯수 default=100
    n_estimators=4000,
    random_state=2019,
    num_boost_round=5000, 
    early_stopping_rounds=4000,
    verbose_eval=3000, 
    show_stdv=False,
    max_depth=100, 
    nthread=6,n
)

#prepare fit model with cross-validation
folds = KFold(n_splits=5, shuffle=True, random_state=2019)

for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train)):
    X_train,X_validation=df_train.iloc[trn_idx][train_columns],df_train.iloc[val_idx][train_columns]
    y_train,y_validation=y_reg.iloc[trn_idx],y_reg.iloc[val_idx]
    
    dtrain = xgb.DMatrix(X_train)
    dtarget=xgb.DMatrix(y_train)
    dtest = xgb.DMatrix(X_validation)
    dtest_target=xgb.DMatrix(y_validation)
    
    model.fit(X_train,y_train
              ,eval_set=[(X_train,y_train),(X_validation,y_validation)]
              ,eval_metric='rmse'
              ,verbose=1000)

[0]	validation_0-rmse:643205	validation_1-rmse:690812
[1000]	validation_0-rmse:12396.5	validation_1-rmse:142010
[2000]	validation_0-rmse:623.625	validation_1-rmse:139451
[3000]	validation_0-rmse:46.1613	validation_1-rmse:139167
[3999]	validation_0-rmse:5.75409	validation_1-rmse:139148
[0]	validation_0-rmse:654797	validation_1-rmse:645531
[1000]	validation_0-rmse:12903.4	validation_1-rmse:151260


In [ ]:
predictions=model.predict(df_test[train_columns])

submission = pd.read_csv("sample_submission.csv")
submission['price']=predictions
submission.to_csv('submission.csv',index=False)